# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

Conversation Memoryには、以下のような種類があります。

* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

ConversationBufferMemoryは、しっかり見てはいませんが、他のメモリーの特徴から、全てのやりとりをそのまま記憶する機能だと思います。
そのため、対話が長くなるトークン制限に引っかかると思います。（実際に引っかかるかを調べるのは面倒なのでやりません）

ConversationBufferWindowMemoryは、直近k回分のやりとりを保持します。

ConversationTokenBufferMemoryは、やりとりの回数ではなく、トークン数で記憶を管理します。max_tokensを決めると、対話履歴全体がその中に収まるようにしてくれます。
長い文章を使った対話を行う場合や、対話履歴に他にもプロンプトを含めたい場合などに有効そうですね。

ConversationSummaryMemoryは、max_tokensに収まるように対話履歴の要約を作成してくれます。
直近の流れがわからないので、QAチャットボットなど、一問一答タイプの会話ならいいかもです。

簡単にいくつかのConversation Memoryについて説明しました。
詳しくは、ドキュメントを見てみてください。ナレッジグラフやベクトルストアを使うメモリーもあるようです。
[Memory | 🦜️🔗 Langchain](https://python.langchain.com/docs/modules/memory/)

## ConversationBufferMemory

まずは、一番シンプルなConversation Memoryです。

In [68]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
import warnings
warnings.filterwarnings('ignore')

In [69]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [70]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [71]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

`verbose=True`とすることで、ConversationChainのプロンプトがわかりますね。
```
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: *****
AI: *****
Human: *****
AI: 
```
"以下の会話は、人間とAIの友好的な会話です。AIはおしゃべりで、コンテキストから具体的・詳細な内容を話します。AIは質問の答えを知らない場合、正直のそう言います。"

チャットボットで扱う場合は、他にも様々な指示をする必要がありそうですね。
例えば、食や献立に関するアドバイスをするチャットボットを試そうとしたときは、いきなり適当なレシピを生成し、詳細な説明を出力してしまいました。
そのため、「レシピ提案をしないでください。」という文言をプロンプトにに入れなければいけませんでした。

In [72]:
conversation.predict(input="What is 1+1?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [73]:
conversation.predict(input="What is my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

In [74]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Andrew, as you mentioned earlier.


In [75]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Andrew, as you mentioned earlier."}

`memory.load_memory_variables({})`これはどういう機能なんでしょうか？よくわからないし、重要ではなさそうです。

In [76]:
memory = ConversationBufferMemory()

In [77]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [78]:
print(memory.buffer)

Human: Hi
AI: What's up


In [79]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [80]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [81]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

次は、直近k回のやりとりを保存するConversation Memoryです。

In [82]:
from langchain.memory import ConversationBufferWindowMemory

`k=1`と設定すると、直近1回のやりとりのみ保存されていることが確認できます。

In [83]:
memory = ConversationBufferWindowMemory(k=1)               

In [84]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [85]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [86]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [87]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [88]:
conversation.predict(input="What is 1+1?")

'The answer to 1+1 is 2.'

In [89]:
conversation.predict(input="What is my name?")

"I'm sorry, I don't have access to that information. Could you please tell me your name?"

先ほどのConversationBufferMemoryと異なり、最初のやりとりで記憶した名前を覚えていません。

## ConversationTokenBufferMemory

次に、トークン数によって保存するやりとりを決めるConversation Memoryです。

In [90]:
# !pip install tiktoken

In [91]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [92]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [93]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

`max_token_limit=30`としているため、2つ目のAIからの応答以降の対話履歴を保存しています。
`input`と`output`は常にセットなのかと思いましたが、そうではないようですね。

## ConversationSummaryMemory

最後は、トークン数に従って、対話履歴の要約を記憶するConversation Memoryです。

In [94]:
from langchain.memory import ConversationSummaryBufferMemory


わかりやすいようにChatGPTで日本語にしました。

In [95]:
# create a long string
schedule = "8時にあなたの製品チームとのミーティングがあります。\
パワーポイントのプレゼンテーションを準備する必要があります。\
午前9時から12時までは、LangChainプロジェクトに取り組む時間があります。\
LangChainは非常にパワフルなツールなので、作業はスムーズに進むでしょう。\
正午には、イタリアンレストランでお昼ご飯をお客様とともにとります。\
お客様は1時間以上かけて運転して会いに来る予定です。\
最新のAIについて理解するために、最新のLLMデモを見せるために、必ずノートパソコンを持ってきてください。"

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "こんにちは。"}, {"output": "どうしたの？"})
memory.save_context({"input": "なんでもないよ。ただぼーっとしてるだけ。"},
                    {"output": "いいね。"})
memory.save_context({"input": "今日の予定は何があった?"}, 
                    {"output": f"{schedule}"})

In [96]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI in Japanese and the AI responds. The human asks about their schedule for the day and the AI provides a detailed list of tasks, including a meeting with the product team, working on the LangChain project, and having lunch with a client at an Italian restaurant. The AI reminds the human to bring their laptop to show the latest LLM demo to understand the latest AI.'}

In [97]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [98]:
conversation.predict(input="どのようなデモが良いでしょうか？")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI in Japanese and the AI responds. The human asks about their schedule for the day and the AI provides a detailed list of tasks, including a meeting with the product team, working on the LangChain project, and having lunch with a client at an Italian restaurant. The AI reminds the human to bring their laptop to show the latest LLM demo to understand the latest AI.
Human: どのようなデモが良いでしょうか？
AI:

> Finished chain.


'最新のLLMデモです。それは、自然言語処理に基づく言語モデルであり、言語の理解と生成に使用されます。デモでは、LLMがどのように機能するかを示し、その機能を実際に体験することができます。それは非常に興味深いです！'

これは会話風ですが、やっぱり一問一答向けです。「何時から何時にLangChainプロジェクトに取り組みますか？」のように、要約で落ちてしまった情報は復元できません。

In [99]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI in Japanese and the AI responds. The human asks about their schedule for the day and the AI provides a detailed list of tasks, including a meeting with the product team, working on the LangChain project, and having lunch with a client at an Italian restaurant. The AI reminds the human to bring their laptop to show the latest LLM demo to understand the latest AI. The human asks about the demo and the AI explains that it is the latest LLM demo, which is a language model based on natural language processing used for language understanding and generation. The demo showcases how LLM works and allows for an interactive experience, which the AI finds very interesting.'}